#### **Deploy de aplicações**

#### **Observações**

#### **Conteúdo - Bases e Notebook da aula**

Github:  
https://github.com/FIAP/Pos_Tech_DTAT/tree/dd201a034223a16732c6f639b40600b26bd3129c

Como criar ambientes para evitar conflito:  
https://github.com/RicardViana/fiap-data-viz-and-production-models/blob/main/Roteiro%20para%20cria%C3%A7%C3%A3o%20de%20ambiente.pdf

Base Credit Card Approval Predictio do Kaggle:  
https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction

#### **Importação de bibliotecas**

In [1]:
# Importar biblioteca completa
import pandas as pd

# Importar algo especifico de uma biblioteca

#### **Funções (def)**

#### **Aula 1 - Tratando os dados**

#### **Aula 2 - Criando a variável target**

#### **Aula 3 - Machine Learning: Construindo a Pipeline**

#### **Aula 4 - Machine Learning: Escolhendo o Melhor Modelo**

#### **Aula 5 - Criando a aplicação**